# **Membangun Aplikasi LLM: Menjawab Pertanyaan dari Dokumen**

In [1]:
# Mengecek versi semua dependensi yang digunakan
import langchain
import langchain_community
import huggingface_hub
import sentence_transformers
import chromadb
import unstructured
import pdfminer
import pypdf
import pdf2image
import unstructured_inference
import unstructured_pytesseract
import dotenv
import numpy
import bert_score

print("langchain:", langchain.__version__)
print("langchain_community:", langchain_community.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("sentence_transformers:", sentence_transformers.__version__)
print("chromadb:", chromadb.__version__)
print("unstructured:", unstructured.__version__)
print("pdfminer.six:", pdfminer.__version__)
print("pypdf:", pypdf.__version__)
print("pdf2image:", pdf2image.__version__)
print("unstructured_inference:", unstructured_inference.__version__)
print("unstructured_pytesseract:", unstructured_pytesseract.__version__)
print("dotenv:", dotenv.__version__)
print("numpy:", numpy.__version__)
print("bert_score:", bert_score.__version__)


ModuleNotFoundError: No module named 'langchain_community'

## Motivation  

Selama ini, yang kita lihat dari LLM (Large Language Model) adalah bahwa model ini hanya bisa digunakan untuk menghasilkan teks berdasarkan sebuah prompt. Tapi, apakah kita bisa memperluas kemampuan LLM / menggunakan LLM yang sudah dilatih sebelumnya (open-source) untuk tugas lain seperti:

`Diberikan sebuah dokumen, kita ingin bisa mengajukan pertanyaan seperti: Berapa angka pertumbuhan ekonomi?`


## Problems

Kita telah melihat beberapa keterbatasan dari LLM (Large Language Model), seperti:

Pengetahuan yang Terbatas (Ada batas waktu pelatihan / cutoff)

Hal ini menyebabkan LLM bisa / sering mengarang informasi (hallucination), atau membuat informasi yang salah.
Lalu, apa pendekatan untuk mengatasi hal ini?

Berikan model pengetahuan eksternal, sebagai informasi kontekstual tambahan, agar model dapat menghasilkan jawaban yang lebih akurat.

## Approach

Pendekatan ini terinspirasi dari Retrieval-Augmented Generation (RAG), di mana pendekatan ini diterapkan pada Open-Domain Question Answering untuk membantu Language Model menjawab pertanyaan dengan lebih akurat, seperti pengetahuan eksternal seperti Wikipedia.

## Solutions
Kami akan mereplikasi pendekatan tersebut, tetapi tidak sepenuhnya mengadopsinya.

Menambahkan Pengetahuan Eksternal

Mengembalikan Dokumen yang Paling Relevan

Language Model mulai Menjawab Pertanyaan

Above Mentioned Approach is inspired from Retrieval Augmented Generation Paper

<img src="https://cs.stanford.edu/~myasu/blog/assets/img/posts/2023-racm3/retrieval_LM.jpg">

## Tools :     

- Langchain
- Huggingface

## Task Detail

1. Adding Document
2. Preprocessing Document
3. Adding Embedding Model
4. Adding Vector Store
5. Create Retriever
6. Adding Language Model as Generator
5. Begin a Query (Asking something based on document above)  

##Task 1 : Adding Document

Tujuan kita dalam tugas ini adalah menambahkan dokumen eksternal, jadi jenis dokumen/informasi apa yang bisa kita tambahkan?
Bergantung pada LangChain, ada berbagai jenis dokumen yang bisa ditambahkan:

Untuk saat ini, akan menggunakan sumber dokumen yang berformat `*.pdf`.

Untuk memuat dokumen .pdf, ada banyak cara. Dalam kasus ini, diperoleh dari artikel bersumber pada `https://hellosehat.com/parenting/` yang sudah dilakukan web scraping, sehingga akan menggunakan `OnlinePDFLoader`.

**Install All Dependencies**



In [ ]:
!pip install langchain
!pip install unstructured
!pip install pdf2image
!pip install pdfminer.six
!pip install unstructured_inference
!pip install pikepdf
!pip install pypdf
!pip install sentence-transformers
!pip install chromadb
!pip install unstructured_pytesseract
!pip install python-dotenv
!pip install huggingface_hub
!pip install unstructured_inference
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.6/212.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 9.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=ab46045735e3f1772fa9a99d9a4527d7e81a365674f62a426638964a0b218f9e
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bc

kenapa menggunakan langchain?
pada case Document Question Answering akan banyak dokumen eksternal (PDF). Dari file pdf ini akan dibuat dari pdf ke text kalau buat dari scract itu akan lama jadi pakai langchain ini akan mempermudah untuk konsep RAG

In [ ]:
# load first
from langchain.document_loaders import OnlinePDFLoader

In [ ]:
batch_norm_url = "/content/parenting.pdf"
document_loader = OnlinePDFLoader(batch_norm_url)

In [ ]:
!pip install pi_heif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.7 MB/s eta 0:00:00


In [ ]:
# proses pemuatan dokumen text dalam framework seperti LangChain
doc_data = document_loader.load()
# nltk data

In [ ]:
doc_data

Output hidden; open in https://colab.research.google.com to view.

di chunk agar mendapatkan part yang sesuai

##Task 2 : Preprocess Document

Langkah selanjutnya yang kita lakukan adalah membagi dokumen kita menjadi bagian-bagian yang lebih kecil. Kenapa?

Beberapa bagian berguna untuk menjawab pertanyaan kita. Karena itu, kita ingin merepresentasikan dokumen sebagai bagian-bagian kecil, sehingga kita dapat menemukan bagian mana yang relevan dengan pertanyaan atau kueri.

Untuk melakukan chunking, kita dapat menggunakan `RecursiveCharacterTextSplitter` dari LangChain dengan argumen berikut:

1. `chunk_size` : Ukuran maksimum potongan (chunk) yang akan dikembalikan

2. `chunk_overlap` : Jumlah karakter yang tumpang tindih antar potongan

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

text_splits = text_splitter.split_documents(doc_data)


In [ ]:
type(text_splits)

list

In [ ]:
text_splits

[Document(metadata={'source': '/content/parenting.pdf'}, page_content='Kategori: Parenting\n\nJudul: Peran Ayah dan Ibu dalam Keluarga, Apa Bedanya?\n\nIsi Artikel: Orangtua memiliki tanggung jawab yang sama dalam mengasuh anak dan menjalankan\n\nsebuah keluarga. Akan tetapi, apakah peran ayah dan ibu dalam keluarga itu sama? Adakah\n\nperbedaannya? Agar lebih jelas, simak ulasannya di bawah ini. Keluarga menjadi wadah utama\n\nuntuk menyediakan kebutuhan biologis dan kesehatan mental anak. Dalam keluarga yang\n\nharmonis, setiap anggota keluarga wajib menjalankan perannya masing-masing. Lalu, adakah\n\nperbedaan antara peran ayah dan ibu dalam keluarga? Sebuah studi terbaru mengemukakan\n\nbahwa peran ayah dan ibu dalam keluarga itu berbeda. Parenting Research Center mengevaluasi\n\ndata dari 2.600 orangtua. Peneliti juga menyoroti perbedaan peran ayah dan ibu dalam keluarga,\n\nmeliputi banyaknya waktu yang dihabiskan bersama anak-anak dan bagaimana tanggung jawab\n\norangtua yang di

In [ ]:
len(text_splits)

3556

In [ ]:
text_splits[0].page_content

'Kategori: Parenting\n\nJudul: Peran Ayah dan Ibu dalam Keluarga, Apa Bedanya?\n\nIsi Artikel: Orangtua memiliki tanggung jawab yang sama dalam mengasuh anak dan menjalankan\n\nsebuah keluarga. Akan tetapi, apakah peran ayah dan ibu dalam keluarga itu sama? Adakah\n\nperbedaannya? Agar lebih jelas, simak ulasannya di bawah ini. Keluarga menjadi wadah utama\n\nuntuk menyediakan kebutuhan biologis dan kesehatan mental anak. Dalam keluarga yang\n\nharmonis, setiap anggota keluarga wajib menjalankan perannya masing-masing. Lalu, adakah\n\nperbedaan antara peran ayah dan ibu dalam keluarga? Sebuah studi terbaru mengemukakan\n\nbahwa peran ayah dan ibu dalam keluarga itu berbeda. Parenting Research Center mengevaluasi\n\ndata dari 2.600 orangtua. Peneliti juga menyoroti perbedaan peran ayah dan ibu dalam keluarga,\n\nmeliputi banyaknya waktu yang dihabiskan bersama anak-anak dan bagaimana tanggung jawab\n\norangtua yang diberikan secara merata di rumah. Umumnya, ayah dianggap sebagai kepala'

Now, check some of it

In [ ]:
print(text_splits[0].page_content)

Kategori: Parenting

Judul: Peran Ayah dan Ibu dalam Keluarga, Apa Bedanya?

Isi Artikel: Orangtua memiliki tanggung jawab yang sama dalam mengasuh anak dan menjalankan

sebuah keluarga. Akan tetapi, apakah peran ayah dan ibu dalam keluarga itu sama? Adakah

perbedaannya? Agar lebih jelas, simak ulasannya di bawah ini. Keluarga menjadi wadah utama

untuk menyediakan kebutuhan biologis dan kesehatan mental anak. Dalam keluarga yang

harmonis, setiap anggota keluarga wajib menjalankan perannya masing-masing. Lalu, adakah

perbedaan antara peran ayah dan ibu dalam keluarga? Sebuah studi terbaru mengemukakan

bahwa peran ayah dan ibu dalam keluarga itu berbeda. Parenting Research Center mengevaluasi

data dari 2.600 orangtua. Peneliti juga menyoroti perbedaan peran ayah dan ibu dalam keluarga,

meliputi banyaknya waktu yang dihabiskan bersama anak-anak dan bagaimana tanggung jawab

orangtua yang diberikan secara merata di rumah. Umumnya, ayah dianggap sebagai kepala


In [ ]:
len(text_splits[0].page_content.split(' '))

123

Nah, di task 3 ini masuk pada Generator -> untuk dapat dijawab dengan aktual, yang diperoleh dari vector yang sama maka diperlukan adanya embedding model

## Task 3 : Adding Embedding Model

Kembali ke tugas peneliti, yaitu membantu LLM menghasilkan jawaban yang lebih akurat dengan menyediakan pengetahuan eksternal.

Yang kita ketahui adalah, kita memiliki banyak potongan teks dari file PDF kita. Untuk menentukan potongan mana yang berguna, yang perlu kita lakukan adalah mengubah dokumen kita menjadi sebuah vektor.

Bagaimana cara kita melakukannya? Kita dapat menggunakan pretrained language model untuk menyandikan embeddings / vektor dari dokumen kita.

Model bahasa yang mana? Untuk saat ini, kita akan menggunakan embedding open-source dari Hugging Face

- Menggunakan model yang sudah di fine tune terhadap similiaritas contohnya model sentence-bert

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

Untuk saat ini, kita akan menggunakan model embedding dari sentence-transformers.

Kenapa kita tidak langsung menggunakan language model secara langsung? Karena itu lambat.

Secara singkat, Sentence-BERT adalah model berbasis encoder (varian Pretrained Masked Language Model) yang telah di-fine-tune untuk tugas kesamaan teks, sehingga vektor yang dihasilkan telah dioptimalkan untuk kemiripan.

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/tmp/ipython-input-15-1656931266.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public m

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Let's see what it can do

In [ ]:
dir(embedding_model)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_setattr_handl

Untuk menghasilkan representasi vektor dari sebuah teks, kita dapat memanggil fungsi `embed_query()`


In [ ]:
len(embedding_model.embed_query('Kapan Anak Harus Mendapatkan Vaksin Hepatitis'))

768

Kita memiliki embedding dengan dimensi 768.
Peneliti mencontoh merubah potongan dokumen menjadi vektor.

In [ ]:
# query_text itu pertanyaan
query_text = "Kapan Anak Harus Mendapatkan Vaksin Hepatitis"
doc_text = text_splits[2].page_content

In [ ]:
# nah disini pula dia lagi memvectorkan query yang ditanyakan
# ini sudah di tokenisasi jadi tinggal memperoleh hasil vector saja
query_vec = embedding_model.embed_query(query_text)
doc_vec = embedding_model.embed_query(doc_text)

In [ ]:
# Jumlahnya 768
query_vec

[0.06550128757953644,
 -0.039766762405633926,
 -0.010523765347898006,
 -0.06224637106060982,
 -0.016234586015343666,
 0.021634148433804512,
 -0.0001961120724445209,
 0.02428385801613331,
 0.0838598683476448,
 -0.001905254554003477,
 0.044154904782772064,
 0.014452881179749966,
 0.032884009182453156,
 0.0632517859339714,
 -0.007370098028331995,
 -0.04655160382390022,
 0.006234369706362486,
 -0.003359136637300253,
 0.018033524975180626,
 -0.006919350009411573,
 -0.015563370659947395,
 0.007838219404220581,
 -0.016471050679683685,
 0.0466221421957016,
 0.027870988473296165,
 -0.05358746275305748,
 0.013776611536741257,
 -0.03231852129101753,
 0.061342667788267136,
 -0.06465961784124374,
 0.05205908790230751,
 -0.03675460442900658,
 -0.0028075450100004673,
 -0.02418849803507328,
 1.7351418364341953e-06,
 0.013936552219092846,
 0.006849451456218958,
 0.00629598181694746,
 -0.019240105524659157,
 -0.013774466700851917,
 0.03921947628259659,
 -0.056250158697366714,
 -0.01287235040217638,
 -0.

In [ ]:
# Jumlahnya 768
doc_vec

[0.06412966549396515,
 -0.06949018687009811,
 0.013986149802803993,
 -0.030833035707473755,
 -0.019811296835541725,
 0.03355581313371658,
 0.056143295019865036,
 -0.005270304158329964,
 0.03337990120053291,
 0.019238920882344246,
 0.02820054069161415,
 0.01934119686484337,
 0.06544137746095657,
 0.024060076102614403,
 -0.005958422552794218,
 -0.023458726704120636,
 -0.006127448286861181,
 0.023997673764824867,
 0.028812557458877563,
 0.02841513603925705,
 -0.04652173072099686,
 0.02510056458413601,
 -0.018014533445239067,
 0.05081174522638321,
 0.057408738881349564,
 -0.02995712123811245,
 -0.0257729422301054,
 -0.007019192446023226,
 0.018441922962665558,
 -0.1295783966779709,
 0.07879170775413513,
 -0.02077856846153736,
 -0.02718019112944603,
 -0.04653625562787056,
 2.676375743249082e-06,
 -0.013568293303251266,
 0.02976096048951149,
 0.01399932149797678,
 -0.042529355734586716,
 -0.011829450726509094,
 0.017573650926351547,
 -0.01202242262661457,
 -0.015410465188324451,
 0.011800055

In [ ]:
len(doc_vec) == len(query_vec)

True

Sekarang, kita dapat menghitung ukuran kemiripan seperti cosine similarity, karena keduanya sekarang berupa vektor.

In [ ]:
import numpy as np
from numpy.linalg import norm

def cosine_sim(vecA, vecB)  :
  upper = np.dot(vecA, vecB)
  lower = norm(vecA) * norm(vecB)
  return upper / lower

In [ ]:
cosine_sim(query_vec, doc_vec)

np.float64(0.5695743262488789)

Untuk menemukan dokumen yang paling relevan, kita dapat mengukur kemiripan antara vektor kueri dan seluruh vektor dokumen.

- Task 4 ini untuk menyimpan hasil vector dari setiap vector yang sudah dicari tadi dan dapat mencari research secara seamless

## Task 4 : Adding Vector Store

Karena dokumen penelitian ini juga sangat banyak, akan sangat berguna untuk menggunakan vector database.

Karena kita menggunakan LangChain, kita akan menggunakan vector store yang kompatibel dengan LangChain. Salah satunya adalah Chroma DB.


In [ ]:
# import first
from langchain_community.vectorstores import Chroma

Untuk memuat potongan dokumen ke dalam vector store, yang kita butuhkan adalah model embedding dan dokumen (yang sudah di-chunk).

In [ ]:
vector_db = Chroma.from_documents(documents=text_splits,
                                    embedding=embedding_model)

In [ ]:
dir(vector_db)

In [ ]:
help(vector_db.similarity_search)

Peneliti dapat melakukan similarity search dengan memanggil fungsi `similarity_search()`.
Kita juga dapat mengatur jumlah dokumen yang dikembalikan dengan menetapkan argumen `k`

In [ ]:
dir(vector_db)

In [ ]:
result = vector_db.similarity_search(query="Kapan Anak Harus Mendapatkan Vaksin Hepatitis", k=10)

In [ ]:
result_with_score = vector_db.similarity_search_with_score(query="Kapan Anak Harus Mendapatkan Vaksin Hepatitis", k=10)

In [ ]:
result_with_score

In [ ]:
result[0].page_content

In [ ]:
result[-1].page_content

## Task 5 : Create Retriever

Setelah menambahkan vector database, kita dapat mengkonfigurasi retriever. Tujuannya adalah untuk mempermudah pencarian dokumen.

Untuk mengatur retriever, kita cukup memanggil fungsi `as_retriever()`. Kita juga bisa menentukan jumlah dokumen untuk menjawab serta jenis pencariannya (search type)

In [ ]:
retriever = vector_db.as_retriever(search_type="similarity",
                                   search_kwargs={"k": 10})

In [ ]:
# to perform similarity search based on query we can run
retrieved_docs = retriever.invoke("Kapan Anak Harus Mendapatkan Vaksin Hepatitis")

In [ ]:
len(retrieved_docs)

In [ ]:
print(retrieved_docs[0].page_content)

In [ ]:
retrieved_docs[2].page_content

## Task 6 : Adding Language Model as Generator

Langkah selanjutnya adalah menambahkan language model untuk menghasilkan jawaban berdasarkan informasi (dokumen) yang diberikan oleh retriever.

Untuk language model-nya, kita akan menggunakan model gratis dan tentunya open-source dari Hugging Face.

**Adding Huggingface Token**

Menyediakan Hugging Face Hub token

```
HUGGINGFACEHUB_API_TOKEN = <YOUR-TOKEN>
```

In [ ]:
import os
from google.colab import userdata

os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('TOKEN')

Model mana yang sebaiknya kita tambahkan?
Akan lebih baik jika kita menambahkan model berbasis chat atau text generation

In [ ]:
# load llm
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", # selain memanggil model menanggil juga tokenizernya yg sudah di vector
    model_kwargs={'max_token':1000},
    task="text-generation")

check language function

Mari menghasilkan keluaran dari sebuah prompt.

In [ ]:
help(llm.generate)

In [ ]:
# print(llm.generate(prompts=['Kapan Anak Harus Mendapatkan Vaksin Hepatiti']).generations[0][0].text)

## Task 7 : Query

We learn that prompting provide context to language model, since the model in instructed style way

<img src="https://images.openai.com/blob/cf717bdb-0c8c-428a-b82b-3c3add87a600/ChatGPT_Diagram.svg?width=10&height=10&quality=50" width=1000 height=500>
<center>
<a href="https://images.openai.com/blob/cf717bdb-0c8c-428a-b82b-3c3add87a600/ChatGPT_Diagram.svg?width=10&height=10&quality=50">
Reinforcement Learnig for Human Feedback (RLHF) , Source
</a>
</center>

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
input_model = custom_rag_prompt.format(context='Barack Obama was born in Indonesia',question='Where Barack Obama was born ?')

Creating Chain

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = ( # pipeline
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm # Prompt dikirim ke Language Model (Mixtral) untuk menghasilkan jawaban
    | StrOutputParser()
)

**Hasil keluaran yang berbeda untuk sebuah pertanyaan**

Language Model without External Document

In [ ]:
docs = retriever.invoke('Kapan Anak Harus Mendapatkan Vaksin Hepatitis')

In [ ]:
formated = format_docs(docs)

In [ ]:
input_model = custom_rag_prompt.format(context=formated,question='Kapan Anak Harus Mendapatkan Vaksin Hepatitis')

Retriever (Vector Search) , Return Relevant Document Only

In [ ]:
retriever.invoke('Kapan Anak Harus Mendapatkan Vaksin Hepatitis')[2]

**Evaluasi**

In [ ]:
!pip install bert-score

In [ ]:
from bert_score import score

# Contoh jawaban referensi (ground truth)
references = ["Pubertas anak laki-laki biasanya terjadi antara usia 9 hingga 14 tahun."]

# Contoh jawaban yang dihasilkan oleh model (prediction)
candidates = ["Anak laki-laki mengalami pubertas pada usia sekitar 9 hingga 14 tahun."]

# Hitung BERTScore
P, R, F1 = score(candidates, references, lang="id")

# Tampilkan hasil
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall: {R.mean().item():.4f}")
print(f"F1-score: {F1.mean().item():.4f}")

So which one do you think its better in terms of generating result ?